## **Spindle detection**

#### Using Yasa Software

yasa.spindles_detect

yasa.spindles_detect(data, sf=None, ch_names=None, hypno=None, include=1, 2, 3, freq_sp=12, 15, freq_broad=1, 30, duration=0.5, 2, min_distance=500, thresh={'rel_pow': 0.2, 'corr': 0.65, 'rms': 1.5}, multi_only=False, remove_outliers=False, verbose=False)

  

Single or multi-channel data. Unit must be **uV** and shape (n_samples) or (n_chan, n_samples). Can also be a mne.io.BaseRaw, in which case data, sf, and ch_names will be automatically extracted, and data will also be automatically converted from Volts (MNE) to micro-Volts (YASA).


Spindles frequency range. Default is 12 to 15 Hz. Please note that YASA uses a FIR filter (implemented in MNE) with a 1.5Hz transition band, which means that for freq_sp = (12, 15 Hz), the -6 dB points are located at 11.25 and 15.75 Hz.

Broad band frequency range. Default is 1 to 30 Hz.


The minimum and maximum duration of the spindles. Default is 0.5 to 2 seconds.



Detection thresholds:

    'rel_pow': Relative power (= power ratio freq_sp / freq_broad).

    'corr': Moving correlation between original signal and sigma-filtered signal.

    'rms': Number of standard deviations above the mean of a moving root mean square of sigma-filtered signal.

You can disable one or more threshold by putting None instead:

thresh = {'rel_pow': None, 'corr': 0.65, 'rms': 1.5}
thresh = {'rel_pow': None, 'corr': None, 'rms': 3}


In [ ]:
import sys  
sys.path.insert(0, 'D:/Beths/')
import os
import re
import math
import pandas as pd
import numpy as np
import yasa
import mne
from mne.filter import filter_data, resample
import matplotlib.pyplot as plt
from scipy import signal
from tqdm import tqdm

## Import from my files
from data_lfp import mne_lfp_Axona, load_lfp_Axona
from data_pos import RecPos
from bandPower import *

#### **Detection of slow-wave sleep, sharp-wave ripples and multi-unit activity bursts**

Paper: Coordinated Emergence of Hippocampal Replay and Theta Sequences during Post-natal Development: 
[https://www.sciencedirect.com/science/article/pii/S0960982219300065]

The brain states slow-wave sleep (SWS), rapid-eye movement sleep (REM) and awake movement were defined following [22
]. A multitaper power spectral density estimate of the hippocampal local field potential (LFP) was derived for **1.6 s windows** , overlapping by 0.8 s (MATLAB function ‘pmtm’). From this, power in the delta and theta bands were calculated in each window. As theta frequency changes during development [18
], theta and delta peak frequencies were calculated for each session, defined as the peak frequency of the fast Fourier transform of the LFP, in the bands 5-11Hz (theta) and 1.5-4Hz (delta). **Mean running speed for each 1.6 s bin was also estimated. In the absence of EMG recordings, we could not unequivocally discriminate between slow wave sleep and quiet immmobility, we therefore restricted all analyses to epochs termed ‘rest’.**

Rest was defined as epochs with running speed < 2.5cm/s , and theta/delta power ratio < 2 and waking movement as theta/delta power ratio > 2 and speed > 2.5cm/s. 
Sharp-wave ripples were detected by first filtering the LFP in the band 100-250Hz. 
The instantaneous power of the filtered LFP was then estimated by calculating the root mean square over 7ms intervals (MATLAB function ‘envelope’ with option ‘rms’). From all LFPs across tetrodes in the CA1 layer, the LFP whose power estimate had the highest standard deviation was then used to define ripple events, as 100ms windows around the peak power, whenever the power was greater than the 99th percentile of all powers in the trial (approximately equal to 4 standard deviations above the mean). Multi-unit activity (MUA) bursts were defined by binning all spikes from CS cells into 1ms bins and smoothing the resulting binned spike train with a Gaussian kernel (s.d. 10ms). MUA events were then defined as crossing of a threshold defined as 3 standard deviations above the mean of the smoothed spike train, with a duration from 100-750ms. Only MUA bursts which temporally overlapped (even in part) with SWR events were included in the replay analysis.

## **Open Sleep Files**

In [ ]:
df = pd.read_csv('data_scheme.csv')
sleep = df.loc[df.sleep == 1]
sleep_files = df.loc[df.sleep == 1, ['folder', 'filename']].agg('/'.join, axis=1).values

In [ ]:
sleep_files = [file for file in sleep_files if 'awake' not in file.split('_')]
sleep_files = [file for file in sleep_files if 'awake.set' not in file.split('_')]
filenames = [r.strip().split('/')[-1] for r in sleep_files]
sleep = df[df.filename.isin(filenames)]
sleep_files = sleep.loc[sleep.sleep == 1, ['folder', 'filename']].agg('/'.join, axis=1).values

In [ ]:
ax = sleep['rat'].value_counts().plot(kind='bar',
                                    figsize=(9,5),
                                    title="Number of sleep recordings for each animal.")
ax.set_xlabel("Rat ID")
ax.set_ylabel("Frequency of recordings")
plt.show()

In [ ]:
ax = sleep['treatment'].value_counts().plot(kind='bar',
                                    figsize=(9,5),
                                    title="Treatment numberd in sleep animals.")
ax.set_xlabel("Treatment type")
ax.set_ylabel("Frequency of recordings")
plt.show()

#### **Auxiliary functions**

In [ ]:
def mark_moving(file, tresh=2.5): # Mark timestamps were the speed > 2.5 cm/s
    '''Mark movement > treshold on timestamps array (1 for movement, 0 for resting)
    Inputs:
        file(str): filename
        tresh(float): treshold in cm/s
    Returns:
        moving(arr): 1 for movement, 0 for resting
        
    '''
    pos = RecPos(file)
    speed = pos.get_speed()
    moving = np.zeros(len(speed)*5)
    for i in range(0, len(speed)): 
        if speed[i] > tresh:
            moving[5*i:5*i+5] = 1
    return moving

def resting_epochs(moving, epoch_size = 2): 
    ''' Define resting epochs on timestamps
    Inputs:
        moving(arr): treshold moving array
        epoch_size(int): size of epochs in seconds
    Returns:
        rest_stamps(arr): File timestamps 0 non rest, 1 rest
    '''
    result = np.zeros(len(moving))
    window = epoch_size * 250
    for i in range(0, len(moving), window//2):
        if sum(moving[i : i + window]) == 0 :
            result[i : i + window] = 1
    return result
    
def create_events(record, events):
    '''Create events on MNE object
    Inputs:
        record(mne_object): recording to add events
        events_time(2D np array): array 0,1 with same lenght of recording dimension (1, lengt(record))
    output: 
    record(mne_object): Record with events added
    '''
    events = np.reshape(events, (1,-1))
    try:
        assert len(record.times) == events.shape[1]
        stim_data = events
        info = mne.create_info(['STI'], record.info['sfreq'], ['stim'])
        stim_raw = mne.io.RawArray(stim_data, info)
        record.add_channels([stim_raw], force_update_info=True)
    except AssertionError as error:
        print(error)
        print('The lenght of events needs to be equal to record lenght.')
    return record

def bandpower(data, low, high, window_sec = 2, sf = 250, relative_power = True):
 
    from scipy.signal import welch
    from scipy.integrate import simps
    scale = 1000 
    lfp_samples = data * scale
    _filter = [1.5, 40]

    # Compute the modified periodogram (Welch)
    nperseg = int(window_sec * sf)
    freqs, psd = welch(lfp_samples, sf, nperseg=nperseg)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's
    # rule)
    bp = simps(psd[idx_band], dx=freq_res)

    idx_band = np.logical_and(freqs >= _filter[0], freqs <= _filter[1])
    tp = simps(psd[idx_band], dx=freq_res)

    if relative_power:
        return bp / tp
    return bp


def bandpower_ratio(data, win_sec=2):
   
    b1 = bandpower(data, 1., 4., win_sec)
    b2 = bandpower(data, 5., 11., win_sec)
    if b1 == b2:
        print("Error!")
    bp = b1 / b2
    return bp

def resting_periods(file, speed_tresh = 2.5, win_sec=2): 
    ''' Define resting epochs on timestamps
    Inputs:
        file(str): file to be calculated
        speed_tresh(float): minimum treshold speed
        win_sec(int): window in seconds to be calculated periods
    Returns:
        file_resting(arr): File timestamps 0 non rest, 1 rest for each channel
    '''
    file_resting = []
    moving = mark_moving(file, speed_tresh)
    mne_file = mne_lfp_Axona(file)
    for channel in tqdm(range(0, len(mne_file.info['ch_names']))):
        data = mne_file.get_data()
        result = np.zeros(len(moving))
        window = win_sec * 250
        for i in range(0, len(moving) - 250, window//2):
            bp = bandpower_ratio(data[channel][i : i + window], win_sec)
            if sum(moving[i : i + window]) == 0 and bp < 2:
                result[i : i + window] = 1
        file_resting.append(result)
    return np.asarray(file_resting)

#### **Plot time distribuition for filtered files times by movement and by movement and relative power**

#### **Divide files into treatment groups**

In [ ]:
musc = sleep.loc[sleep.treatment == 'muscimol']
control = sleep.loc[sleep.treatment == 'Control']
lesion = sleep.loc[sleep.treatment == 'lesion']
print(f'Muscimol: {len(musc)} recordings in {len(musc.rat.unique())} animals')
print(f'Control: {len(control)} recordings in {len(control.rat.unique())} animals')
print(f'Lesion: {len(lesion)} recordings in {len(lesion.rat.unique())} animals')
msc_files = musc.loc[musc.treatment == 'muscimol', ['folder', 'filename']].agg('/'.join, axis=1).values
cnt_files = control.loc[control.treatment == 'muscimol', ['folder', 'filename']].agg('/'.join, axis=1).values
les_files = lesion.loc[lesion.treatment == 'muscimol', ['folder', 'filename']].agg('/'.join, axis=1).values

### **Calculate spindles**

In [ ]:
data = mne_lfp_Axona(file)

In [ ]:
f_array = resting_periods(file, speed_tresh = 2.5, win_sec=2)

In [ ]:
plt.plot(f_array[0])

In [ ]:
ax = plt.figure(figsize=(12, 4))
plt.specgram(data[0:50000], NFFT=500, Fs=250., noverlap=250  ,cmap='jet_r')
plt.show()

In [ ]:
test_data = msc_files[0]
data = mne_lfp_Axona(test_data)
sp = yasa.spindles_detect(data, 250., ch_names=['ch_1'], thresh={'rel_pow': 0.2, 'corr': 0.65, 'rms': 2.5}, freq_sp= (12, 15), multi_only=True, verbose='error')

In [ ]:
df = sp.summary()

In [ ]:
ch1 = df.loc[df.Channel == 'ch_1']
ch2 = df.loc[df.Channel == 'ch_2']

In [ ]:
print(f'Ch1 oscilations mean: {ch1.Oscillations.mean():.2f}, std: {ch1.Oscillations.std():.2f}')
print(f'Ch2 oscilations mean: {ch2.Oscillations.mean():.2f}, std: {ch2.Oscillations.std():.2f}')

In [ ]:
ch4 = df.loc[df.Channel == 'ch_4']
ch5 = df.loc[df.Channel == 'ch_5']

In [ ]:
print(f'Ch4 oscilations mean: {ch4.Oscillations.mean():.2f}, std: {ch4.Oscillations.std():.2f}')
print(f'Ch5 oscilations mean: {ch5.Oscillations.mean():.2f}, std: {ch5.Oscillations.std():.2f}')

In [ ]:
def process_spindles(files, channel_names = ['ch_1']):
    ''''''
    spindles_list = []
    for file in files:
        data = mne_lfp_Axona(file)
        sp = yasa.spindles_detect(data, 250., ch_names=channel_names, thresh={'rel_pow': 0.2, 'corr': 0.65, 'rms': 2.5}, freq_sp= (12, 15), multi_only=True, verbose='error')
        df = sp.summary()
        df.loc[df.]
        
    return spindles_list


In [ ]:
sps = process_spindles(msc_files, channel_names = ['ch_1'])

In [ ]:
sps[0].summary()

In [ ]:
# Load the data
data = mne_lfp_Axona(sleep_files[13])
data = data.filter(l_freq=10, h_freq = 16,  method='fir',verbose=0).copy()

In [ ]:
data.info


**Spindle Detection and Verification** [https://academic.oup.com/sleep/article/39/5/1069/2454043?login=true#125103895]

We developed an algorithm to automatically detect sleep spindle events in line with previous studies.(20,39) LFP raw data was resampled to 1,000 Hz, band-pass filtered (using a zero-phase, second order, Infinite Impulse Response Butterworth filter) between 10–16 Hz and the instantaneous amplitude was extracted via the Hilbert transform. Then, two thresholds were set relatively to the mean band-pass signal during NREM sleep: (1) a “detection threshold” (+2 SD above the mean) identified events as potential spindles, and (2) a “noise threshold” (+0.2 SD above the mean) was used to define the start and end of sleep spindle event. To verify specificity for sleep spindles (versus broadband power increases), we excluded any putative spindle event whose instantaneous amplitude in a control frequency band (20–30 Hz) exceeded a predefined threshold of +4.5 SD above the mean. Finally an event qualified as a spindle if its duration was between 0.5 and 2.5 sec. It should be emphasized that the specific parameters of the spindle detection algorithm (e.g., frequency range, filter settings, thresholds) were optimized after extensive visual inspection to minimize false detections, and a wide range of parameters yielded similar detections and overall results (data not shown). Power spectral density (Figure 2B) was computed on 350-msec time windows centered on spindle peaks or random time-intervals in NREM sleep.

#### **Calculate Welch's PSD**

The gold-standard and most widely-used method to calculate the power spectral density (PSD) of EEG data is the Welch's sliding periodogram, which is implemented in SciPy.

The Welch's method improves the accuracy of the classic periodogram. The reason is simple: EEG data are always time-varying, meaning that if you look at a 30 seconds of EEG data, it is very (very) unlikely that the signal will looks like a perfect sum of pure sines. Rather, the spectral content of the EEG changes over time, constantly modified by the neuronal activity at play under the scalp. Problem is, to return a true spectral estimate, a classic periodogram requires the spectral content of the signal to be stationnary (i.e. time-unvarying) over the time period considered. Because it is never the case, the periodogram is generally biased and contains way too much variance. By averaging the periodograms obtained over short segments of the windows, the Welch's method allows to drastically reduce this variance. This comes at the cost, however, of a lower frequency resolution. The frequency resolution in Welch's method is defined by the window length, such that $F = 1 / t$, with $t$ being the window length in seconds. In other words, a 4-sec sliding window will give a frequency resolution of 1 / 4 = 0.25 Hz.

How do we define the optimal window duration then? A commonly used approach is to take a window sufficiently long to encompasses at least two full cycles of the lowest frequency of interest. So for instance, if our lowest frequency of interest is 1 Hz so we will choose a window of 2 / 1 = 2 seconds.

Note that in the code below, I use a median average of all the resulting sliding windows. Indeed, this may result in less biased and more accurate PSD as explained in this 